In [42]:
import tkinter as tk
import pymongo

# Connexion à la base de données MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]

users = db['users']
users_labeled = db['users_labeled']
user_scaled_collection = db["users_scaled"]

# Variables globales pour suivre l'état actuel
users_ids_labeled = [doc['_id'] for doc in users_labeled.find({}, {'_id': 1})]
documents = users.find({'_id': {'$nin': users_ids_labeled}})
current_document = None

# Seuils de suspicion
min_values_suspicious = {
    'hashtag_frequency': 5,
    'favourites_count': 60_000,
    'friends_count': 6_000,
    'followers_count': 9_000,
    'ratio_friends_followers': 10,
    'tweet_frequency': 70,
    'Ap': 0.02,
    'ratio_sensitive_tweets': 0,
    'ratio_punctuation_tweets': 0.5,
    'visibility': 0.5
}

# Fonction pour passer au document suivant
def document_suivant():
    global current_document
    current_document = next(documents, None)
    if current_document:
        update_label_style()

# Fonction pour copier le document actuel avec le label approprié
def copier_document(valeur_label):
    if current_document:
        document_copie = dict(current_document)
        document_copie['label'] = valeur_label
        users_labeled.insert_one(document_copie)
        document_suivant()

# Fonction pour mettre à jour le style de l'étiquette
def update_label_style():
    label.delete('1.0', 'end')  # Effacer le contenu précédent
    for key, value in current_document.items():
        line = f"{key}: {value}"
        if key in min_values_suspicious and value > min_values_suspicious[key]:
            label.insert('end', line + '\n', 'red')
        else:
            label.insert('end', line + '\n')

# Création de l'interface graphique
root = tk.Tk()
root.title("Copier des documents MongoDB")

# Configuration du style des boutons
button_style = {'font': ('Arial', 20, 'bold'), 'width': 12, 'height': 3, 'foreground': 'white'}

# Étiquette pour afficher les valeurs des attributs
label = tk.Text(root, font=('Arial', 14))
label.pack(pady=20)

# Définir un tag pour les lignes en rouge
label.tag_configure('red', foreground='red')

# Boutons pour le label
button_0 = tk.Button(root, text="0 - Normaux", command=lambda: copier_document(0), **button_style)
button_0.pack(side="left", padx=10)
button_1 = tk.Button(root, text="1 - Atypique", command=lambda: copier_document(1), **button_style)
button_1.pack(side="left", padx=10)

# Modifier la couleur de fond du bouton 0 en vert
button_0.configure(background='green')

# Modifier la couleur de fond du bouton 1 en rouge
button_1.configure(background='red')

# Passer au premier document
document_suivant()

# Démarrage de la boucle principale de l'interface graphique
root.mainloop()
print('Session de labélisation terminée')

# Standardisation
print("On standardise les données par rapport à celles déjà standardisé sur l'ensemble des users")
pipeline = [
    {
        '$merge': {
            'into': 'users_labeled',
            'on': '_id',
            'whenMatched': 'merge',
            'whenNotMatched': 'discard'
        }
    }
]

user_scaled_collection.aggregate(pipeline)
print('FIN')

Session de labélisation terminée
On standardise les données par rapport à celles déjà standardisé sur l'ensemble des users
FIN
